# Trading Manager for Crypto

This notebook allow to comunicate with the services running in real time. All the comunications it is with MQ messages.

In [17]:
from smartbots.database_handler import Universe
from smartbots import conf
from smartbots import events
import datetime as dt
from smartbots.brokerMQ import Emit_Events
import time

# Conection to the database
store = Universe()
lib_petitions = store.get_library('petitions')
# Conection to broker mq
emiter = Emit_Events()

def get_data_petition(name):
    for i in range(10):
        time.sleep(1)
        # check if name in list_symbols
        list_symbols = lib_petitions.list_symbols()
        if name in list_symbols:
            return lib_petitions.read(name).data
        time.sleep(5)



## Analice Realtime positions from the portfolio VS Realtime position in Exchange
It sends a petition msg to the portfolio service to get the realtime positions.

In [ ]:
# Create the petition
name_to_saving = f'positions_{dt.datetime.now().strftime("%Y%m%d%H%M%S")}'
function_to_run='get_saved_values_strategies_last' #get_saved_values_strategy  
petition_pos = events.Petition(datetime=dt.datetime.now(), function_to_run=function_to_run,
                               name_to_saving=name_to_saving)

emiter.publish_event('petition', petition_pos)


In [71]:
# Compute diference between broker and portfolio
diference = {}
for c in broker_pos.keys():
    diference[c] =  round(agregate[c] - broker_pos[c],4)
print(diference)

{'USDT': -193.6037, 'ETH': 0.1}


In [72]:
# Create order to close diference
n= 0
for t in tickers:
   d = diference[t.split('-')[0]]
   if abs(d) > 0.01:
       if d > 0:
           action = 'buy'
       else:
           action = 'sell'
       order_id_sender =f' {0}_{n}_{dt.datetime.utcnow().strftime("%Y%m%d%H%M%S")}'
       order = events.Order(datetime=dt.datetime.utcnow(),
                            ticker=t, action=action,
                            price=0, quantity=abs(d), type='market', sender_id=0,
                            order_id_sender=order_id_sender)
       emiter.publish_event('order', order)
       n += 1

